# **Importing Libraries**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [ ]:
import re
import nltk
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**loading the IMDB dataset**

In [ ]:
data = load_dataset('imdb')
train_data = data['train']
test_data = data['test']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
train = [x['text'] for x in train_data]
test = [x['text'] for x in test_data]
review = train + test
sentiment = [x['label'] for x in train_data] + [x['label'] for x in test_data]

# **Preprocessing the imdb data in a single function**

In [ ]:
stopwords_list = set(stopwords.words('english'))

tag_re = re.compile(r"<[^>]+>")

def remove_tags(text):
    return tag_re.sub('', text)

def preprocess(text):
    text = text.lower()

    # Remove HTML tags
    text = remove_tags(text)

    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove single characters
    text = re.sub(r"\b[a-zA-Z]\b", '', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords_list]

    return ' '.join(words)

**Applying the preprocess function at imdb text data**

In [ ]:
x = []
for i in review:
  x.append(preprocess(i))

# **Lemmatizing the text**

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
lematizer = WordNetLemmatizer()
def lematize(text):
  token = word_tokenize(text)
  lematized = [lematizer.lemmatize(word) for word in token]
  return ' '.join(lematized)

In [ ]:
x_lematized = []
for i in x:
  x_lematized.append(lematize(i))

**Using Term Frequency-Inverse Document Frequency (TF-IDF) converting the text into data**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x_lematized)

In [ ]:
for i in range(5):
  print(x[i])
  print('_'*1000)

  (0, 147352)	0.06887165744892833
  (0, 40261)	0.43163979086357285
  (0, 191879)	0.05103366923832112
  (0, 170093)	0.06341512274416986
  (0, 36485)	0.0915987965354543
  (0, 173807)	0.08213501373585433
  (0, 64729)	0.06237970433627787
  (0, 146763)	0.05638557253809167
  (0, 5489)	0.03089083548910607
  (0, 79493)	0.05438797765084294
  (0, 157091)	0.1178250011423481
  (0, 40410)	0.09316446837947953
  (0, 57540)	0.034517189411346454
  (0, 184650)	0.058776008635458406
  (0, 55739)	0.07623589830465599
  (0, 37765)	0.05643781951133665
  (0, 178763)	0.06949279056901347
  (0, 60877)	0.042142107624914556
  (0, 63298)	0.08036879938586203
  (0, 35913)	0.13075712669574147
  (0, 36480)	0.07984120637884981
  (0, 145020)	0.08560638910590448
  (0, 156655)	0.02731785790963409
  (0, 119242)	0.12967365581816623
  (0, 136334)	0.067683669217308
  :	:
  (0, 93972)	0.04926885352501424
  (0, 66583)	0.07789364112799925
  (0, 63981)	0.10843783114239668
  (0, 34260)	0.10514179186104686
  (0, 63702)	0.057751311774

In [ ]:
y=sentiment

**Splitting the processed data into train test split for model training**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

**Training the model using LogisticRegression model**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred))
print("Recall: ",recall_score(y_test,y_pred))
print("F1 Score: ",f1_score(y_test,y_pred))

Accuracy:  0.8912
Precision:  0.880449792858552
Recall:  0.9025278058645096
F1 Score:  0.8913521070501298


**Tuning Hyperparameters of logisticregression model**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ['l2', 'l1'],
    "solver": ['liblinear', 'saga'],
    "max_iter": [100, 200, 300, 500]
}
random = RandomizedSearchCV(lr, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
random.fit(x_train,y_train)

best_model = random.best_estimator_
y_pred = best_model.predict(x_test)
# Evaluate the model
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')


print(f"Best Parameters: {random.best_params_}")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'C': 10}
Accuracy: 0.8820
Precision: 0.8821
Recall: 0.8820


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**Training the model using naive bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(x_train,y_train)
y_pred=nb.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred))
print("Recall: ",recall_score(y_test,y_pred))
print("F1 Score: ",f1_score(y_test,y_pred))

Accuracy:  0.8672
Precision:  0.8741982205669356
Recall:  0.8543983822042467
F1 Score:  0.8641849048885253


**Tuning Hyperparameters of Naive bayes model**

In [ ]:
params={
        "alpha": [0.1, 0.5, 1.0, 2.0, 10.0],
        "fit_prior": [True, False],
        "class_prior": [None, [0.5, 0.5]]  # For balancing classes
    }
random = RandomizedSearchCV(nb, params, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
random.fit(x_train,y_train)

best_model = random.best_estimator_
y_pred = best_model.predict(x_test)
# Evaluate the model
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')


print(f"Best Parameters: {random.best_params_}")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'fit_prior': True, 'class_prior': None, 'alpha': 1.0}
Accuracy: 0.8672
Precision: 0.8674
Recall: 0.8672


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred))
print("Recall: ",recall_score(y_test,y_pred))
print("F1 Score: ",f1_score(y_test,y_pred))

**Traing the model using support vector machine**

In [ ]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred))
print("Recall: ",recall_score(y_test,y_pred))
print("F1 Score: ",f1_score(y_test,y_pred))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grids ={
     "C": [0.01, 0.1, 1, 10, 100],
      "kernel": ['linear', 'rbf', 'poly', 'sigmoid'],
      "gamma": ['scale', 'auto', 0.01, 0.1],
      "degree": [2, 3, 4],  # Only for 'poly' kernel
      "coef0": [0, 0.1, 1],  # Only for 'poly' and 'sigmoid'
      "class_weight": ['balanced', None]
    }
random = RandomizedSearchCV(svc, param_grids, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
random.fit(x_train,y_train)

best_model = random.best_estimator_
y_pred = best_model.predict(x_test)
# Evaluate the model
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')


print(f"Best Parameters: {random.best_params_}")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

**Tuning hyperparameters for Support vector machine model**